<a href="https://colab.research.google.com/github/xuxoramos/nosql-4-ds/blob/gh-pages/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>